## Template of the Test Code
The test code template is shown below:

In [1]:
import numpy as np
from sklearn import metrics

def test(model, test_dataset_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    y_true = []
    y_pred = []
    test_dataset = data_loader(test_dataset_path)
    
    for img, label in test_dataset:
        # Please make sure that the "pred" is binary result
        output = model(img.unsqueeze(dim=0).to(device))
        pred = np.argmax(output.detach().to('cpu'),axis=1).item()
        
        y_true.append(label)
        y_pred.append(pred)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    Accuracy = metrics.accuracy_score(y_true, y_pred)

    return Accuracy

Please make sure that the final "y_pred" should be a binary array. This is because "metrics.accuracy_score(y_true, y_pred)" calculates the accuracy between two label indicator arrays. For example:

In [2]:
y_pred = np.array([0, 0, 1, 1])
y_true = np.array([0, 1, 0, 1])
metrics.accuracy_score(y_true, y_pred)

0.5

## A demo of "data_loader" and "model"
A demo of "data_loader" and "model" is shown below. You need to construct your own "data_loader" and "model".

In [3]:
import torch
from torch.utils.data import Dataset
import os
from PIL import Image
import torchvision.transforms as transforms

class data_loader(Dataset):
    def __init__(self, data_dir):
        # Get Image File Names
        real = os.path.join(data_dir, '0_real')  # directory of files
        fake = os.path.join(data_dir, '1_fake')

        file_names_real = os.listdir(real)  # get list of images in that directory
        self.full_filenames_real = [os.path.join(real, f) for f in file_names_real]  # get the full path to images
        file_names_fake = os.listdir(fake)  # get list of images in that directory
        self.full_filenames_fake = [os.path.join(fake, f) for f in file_names_fake]  # get the full path to images
        self.full_filenames = self.full_filenames_real + self.full_filenames_fake

        self.labels_real = [0 for filename in file_names_real]
        self.labels_fake = [1 for filename in file_names_fake]
        self.labels = self.labels_real + self.labels_fake

        self.transform = transforms.Compose([
                                            transforms.RandomResizedCrop(size=(64, 64)),
                                            transforms.ToTensor(),
                                            ])
        
    def __len__(self):
        return len(self.full_filenames_real) + len(self.full_filenames_fake) # size of dataset

    def __getitem__(self, idx):
        # open image, apply transforms and return with label
        image = Image.open(self.full_filenames[idx])  # Open Image with PIL
        image = self.transform(image) # Apply Specific Transformation to Image
        return image, self.labels[idx]


import torch.nn as nn
import torch.nn.functional as F

# Neural Network
class Network(nn.Module):
    
    # Network Initialisation
    def __init__(self, params):
        
        super(Network, self).__init__()
    
        Cin,Hin,Win=params["shape_in"]
        init_f=params["initial_filters"] 
        num_fc1=params["num_fc1"]  
        num_classes=params["num_classes"] 
        self.dropout_rate=params["dropout_rate"] 
        
        # Convolution Layers
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
        h,w=findConv2dOutShape(Hin,Win,self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv3)
        self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv4)
        
        # compute the flatten size
        self.num_flatten=h*w*8*init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self,X):
        
        # Convolution & Pool Layers
        X = F.relu(self.conv1(X)); 
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)

        X = X.view(-1, self.num_flatten)
        
        X = F.relu(self.fc1(X))
        X=F.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return F.log_softmax(X, dim=1)
    
def findConv2dOutShape(hin,win,conv,pool=2):
    # get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation

    hout=np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    wout=np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

    if pool:
        hout/=pool
        wout/=pool
    return int(hout),int(wout)

# Neural Network Predefined Parameters
params_model={
        "shape_in": (3,64,64), 
        "initial_filters": 8,    
        "num_fc1": 100,
        "dropout_rate": 0.25,
        "num_classes": 2}

# Create instantiation of Network class
cnn_model = Network(params_model)

# define computation hardware approach (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = cnn_model.to(device)

## What TAs do
In this way, TAs can get the test result by only replacing the “test_dataset_path”.

In [4]:
test_dataset_path = 'path_to_test_set'
test(model, test_dataset_path)

0.5